In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os/
import pathlib as pl
%matplotlib inline 
engine=create_engine('postgresql://postgres:++++++++++.@+++++++++/++++++', max_overflow=20)

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())
    

Front end queries 

Relation between average speed and passengers amount, and the quantity of routes with theses characteristics

In [ ]:
query = '''
SELECT secuenciarecorrido, idruta, avg(CANTPASAJEROSSUBEN) as pasajeros, avg(velocidad) as vel_promedio
    FROM detalles_recorrido WHERE recorridofinalizado = 'S'
    group by secuenciarecorrido, idruta;
    '''
df = runQuery(query)   
df.head(10)
sns.displot(x='pasajeros',y='vel_promedio',data=df[df['pasajeros']<100],binwidth=(10, 10),cbar=True)

Second: Companies whose average speed during the trip is greater than 60 km/h 

In [ ]:
query = '''
with speed_dist as (select r.secuenciarecorrido, r.idruta,  r.velocidad, v.idempresa , v.nombres  
	from detalles_recorrido r
  left join vehiculos v on r.idvehiculo  = v.idvehiculo
where r.velocidad > 60) 
select nombres as nombre_empresa, count(distinct secuenciarecorrido) as num_recorridos,avg(velocidad) as vel_promedio
from speed_dist
group by 1
order by vel_promedio desc
'''
df = runQuery(query)   
chart=sns.barplot(x="nombre_empresa", y="num_recorridos", data=df.head(10))
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

Third: Passengers demand by route for a specific date

In [ ]:
query = '''
with datos as (
select distinct 
  idruta as ruta, 
  cantpasajerossuben 
from detalles_recorrido 
where ultimafecharegistro='2019-02-06'
) 
select 
  ruta, 
  sum(cantpasajerossuben) as total_demanda
from datos 
group by 1
order by 2 desc
'''
df = runQuery(query) 
chart=sns.barplot(x="ruta", y="total_demanda", data=df.head(10))
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

Fourth: Average speed per route

In [ ]:
query = '''
SELECT idruta, avg(velocidad) as vel_promedio
    FROM detalles_recorrido WHERE recorridofinalizado = 'S'
    group by idruta;
'''
df = runQuery(query)
sns.displot(df, x="vel_promedio", kind="kde", fill=True)

EDA


In [ ]:
#Register count per trip.
ids_recorridos = pd.read_sql("""
SELECT secuenciarecorrido, COUNT(*) AS Conteo
FROM recorrido
GROUP BY secuenciarecorrido;
""",con = ruta)
ids_recorridos["conteo"].mean()

In [ ]:
#Register count per type of trip (finished, unfinished)
recorridos_finalizados = pd.read_sql("""
SELECT recorridofinalizado, secuenciarecorrido, COUNT(*) AS Conteo
FROM recorrido
GROUP BY secuenciarecorrido,recorridofinalizado;
""",con = ruta)
recorridos_finalizados = recorridos_finalizados.groupby("recorridofinalizado").agg("count")[["conteo"]].reset_index()
recorridos_finalizados

In [ ]:
#Vehicles count
vehiculos_conteo = pd.read_sql("""
SELECT idvehiculo, COUNT(*) AS Conteo
FROM recorrido
GROUP BY idvehiculo;
""",con = ruta)
vehiculos_conteo.shape
vehiculos_conteo["conteo"].mean()

In [ ]:
#Register count per company
import plotly.express as px

empresas_conteo = pd.read_sql("""
SELECT idempresa, COUNT(*) AS Conteo
FROM recorrido
GROUP BY idempresa;
""",con = ruta)
empresas_conteo.shape
empresas_conteo["conteo"].mean()
fig = px.bar(empresas_conteo, x='idempresa', y='conteo',
            labels = {'idempresa':'IDcompany',"conteo":"Count"},
             title = "Distribution of register by company")
fig.show()

In [ ]:
#Register count per route 
rutas_conteo = pd.read_sql("""
SELECT codigoruta, COUNT(*) AS Conteo
FROM recorrido
GROUP BY codigoruta;
""",con = ruta)
rutas_conteo.shape #cantidad de filas son la cantidad de rutas únicas
rutas_conteo["conteo"].mean() #Cantidad de registros promdio por ruta 


In [ ]:
#Register count per date 
df_fechas_conteo = pd.read_sql("""
SELECT DATE(fecharegistro) AS Date, COUNT(*) AS Count
FROM recorrido
GROUP BY Date;
""",con = ruta)
df_fechas_conteo['date'] = pd.to_datetime(df_fechas_conteo['date'], format="%Y-%m-%d") #Variable como fecha
df_otrosmeses = df_fechas_conteo[(df_fechas_conteo["date"]<="2020-09-01")&(df_fechas_conteo["date"]>="2020-01-01")] #filtro de data para meses antes de septiembre
septiembre = df_fechas_conteo[(df_fechas_conteo["date"]>="2020-09-01")&(df_fechas_conteo["date"]>="2020-01-01")] #Filtro para septiembre
#se dividen en dos df poque la cantidad de datos de septiembre es demasiado grande para colocarlo en comparación con los demás meses (Septiembre tiene millones y los otros meses tienen miles)
# A partir de aquí correr en chunks independientes
#grafico para la base de septiembre
fig = px.line(septiembre, x="date", y="count", title='Count register in september')
fig.show()
#grafico para la base para los otros meses
fig = px.line(df_otrosmeses, x="date", y="count", title='Count register other months')
fig.show()

In [ ]:
##Top 20 of Register count of finished trips, whose speed is greater than 60
df7 = pd.read_sql("""
    SELECT COUNT(secuenciarecorrido) exceso,secuenciarecorrido
        from recorrido WHERE recorridofinalizado = 'S' AND velocidad > 60 
        group by secuenciarecorrido
        ORDER BY exceso DESC
        LIMIT 20;
""",con = ruta)
ids_recorridos["conteo"].mean()
chart=sns.barplot(x="secuenciarecorrido", y="exceso", data=df7)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

In [ ]:
#Average registers of finished trips, whose speed is greater than 60
countexcesosAvg = pd.read_sql("""
SELECT AVG(f.exceso) 
FROM (
    SELECT COUNT(secuenciarecorrido) exceso,secuenciarecorrido
        from recorrido WHERE recorridofinalizado = 'S' AND velocidad > 60 
        group by secuenciarecorrido
        ORDER BY exceso DESC
        ) f;
""",con = ruta)
countexcesosAvg

In [ ]:
#Companies with trips with a speed greater than 60
df8 = pd.read_sql("""
select count( distinct  g.secuenciarecorrido), g.idempresa, g.nombres from
 (select r.secuenciarecorrido, r.codigoruta, (r.subendelantera+r.subentrasera) as suben, r.velocidad, v.idempresa , v.nombres  
	from recorrido r, vehiculos v, empresas e where r.idvehiculo  = v.idvehiculo and r.velocidad > 60) g
group by g.idempresa, g.nombres order by count desc
""",con = ruta)
chart=sns.barplot(x="nombres", y="count", data=df8.head(10))
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

In [ ]:
#average passenger demand of finished trips
promediopasajeros = pd.read_sql("""
select avg(f.total) from (
	Select  sum(g.suben) as total, g.secuenciarecorrido 
		from (
			SELECT secuenciarecorrido, codigoruta, (subendelantera+subentrasera) as suben, 
			DATE_PART('dow',fecharegistro::TIMESTAMP) as dia 
			from datafilter WHERE recorridofinalizado = 'S' ) 
		g group by  g.secuenciarecorrido) f;

""",con = ruta)

In [ ]:
#Demand per day of the week 
sumaDemandaday = pd.read_sql("""
Select  sum(g.suben) as total, g.dia 
from (
	SELECT secuenciarecorrido, codigoruta, (subendelantera+subentrasera) as suben, 
	DATE_PART('dow',fecharegistro::TIMESTAMP) as dia 
	from recorrido WHERE recorridofinalizado = 'S' ) 
g group by  g.dia order by total desc;

""",con = ruta)
sumaDemandaday


In [ ]:
#Average capacity per company

query = pd.read_sql("""
with company_info as (
	select distinct 
		e.nombres as company_name, 
		r.idvehiculo, 
		v.cappasajeros as capacity
	from recorrido r
	left join empresas e on r.idempresa = e.idempresa
	left join vehiculos v on r.idvehiculo = v.idvehiculo
	where 
		fecharegistro::date between '2020-09-21' and '2020-09-25'
)
select 
	company_name, 
	round(avg(capacity)) as avg_capacity
from company_info
group by 1
order by avg_capacity desc 

""",con = ruta)
average_capacity_per_company = query
average_capacity_per_company

In [ ]:
#Average routes number per company
query = pd.read_sql("""
with number_of_routes_per_day as (
	select 
		fecharegistro::date as register_date, 
		e.nombres as company_name, 
		count(distinct codigoruta)	as total_routes 
	from recorrido r
	left join empresas e on r.idempresa = e.idempresa
	group by 1,2
) 
select 
	company_name, 
	round(avg(total_routes)) avg_route_number
from number_of_routes_per_day 
where 
	register_date between '2020-09-21' and '2020-09-25'
group by 1
order by avg_route_number desc 
limit 10 ;

""",con = ruta)
number_routes_per_company = query
plt.figure(figsize=(10,10))
ax=sns.barplot(y='company_name',x='avg_route_number',data=number_routes_per_company,
            color='blue',alpha=0.5,orient='h')
plt.xlabel('')
plt.ylabel('')
plt.title('Average number of routes per company')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.tight_layout()
plt.savefig('avg_number_of_routes_per_company.png')
plt.figure(figsize=(10,10))
ax=sns.barplot(y='company_name',x='avg_route_number',data=number_routes_per_company,
            color='blue',alpha=0.5,orient='h')
plt.xlabel('')
plt.ylabel('')
plt.title('Average number of routes per company')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.tight_layout()
plt.savefig('avg_number_of_routes_per_company.png')


In [ ]:
#trip count and average speed per company, when speed is greater than 60
query='''
with speed_dist as (select r.secuenciarecorrido, r.idruta,  r.velocidad, v.idempresa , v.nombres  
	from detalles_recorrido r
  left join vehiculos v on r.idvehiculo  = v.idvehiculo
where r.velocidad > 60) 
select nombres as nombre_empresa, count(distinct secuenciarecorrido) as num_recorridos,avg(velocidad) as vel_promedio
from speed_dist
group by 1
order by vel_promedio desc
'''
df = runQuery(query)
df

In [ ]:
#Passenger demand per route 
with datos as (
select distinct 
  secuenciarecorrido,
  idruta as ruta, 
  cantpasajerossuben 
from detalles_recorrido 
where ultimafecharegistro='2019-02-06'
) 
select 
  ruta, 
  sum(cantpasajerossuben) as total_demanda
from datos 
group by 1
order by 2 desc